In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras

Using TensorFlow backend.


In [2]:
(X_train,Y_train),(X_test,Y_test) = keras.datasets.cifar10.load_data()
X_train = keras.applications.vgg19.preprocess_input(X_train, mode='tf')
X_test = keras.applications.vgg19.preprocess_input(X_test, mode='tf')

In [3]:
G = keras.applications.VGG19(include_top=False)
G_outputs = [l.output for l in G.layers if isinstance(l, keras.layers.MaxPool2D)]
G_outputs = [keras.layers.GlobalAvgPool2D()(l) for l in G_outputs]
G_outputs = keras.layers.Concatenate()(G_outputs)
G = keras.Model(G.input, G_outputs)
G.trainable = False

In [4]:
X_train_g = G.predict(X_train, batch_size=128, verbose=1)
X_test_g = G.predict(X_test, batch_size=128, verbose=1)

10000/10000 [==============================] - 19s 2ms/step


In [5]:
X = X_input = keras.layers.Input(X_train_g.shape[1:])
X = keras.layers.BatchNormalization(center=False, scale=False)(X)
X = keras.layers.Dense(max(np.max(Y_train),np.max(Y_test))+1, activation='softmax')(X)
M = keras.Model(X_input, X)
M.compile('nadam', 'sparse_categorical_crossentropy', ['acc'])
M.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1472)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1472)              2944      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                14730     
Total params: 17,674
Trainable params: 14,730
Non-trainable params: 2,944
_________________________________________________________________


In [6]:
M.fit(
    X_train_g, Y_train,
    validation_data=(X_test_g,Y_test),
    batch_size=128, epochs=50, callbacks=[
    keras.callbacks.ReduceLROnPlateau(patience=2, verbose=1)
])

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 2s 36us/step - loss: 0.9466 - acc: 0.6780 - val_loss: 0.8192 - val_acc: 0.7222
Epoch 2/50
50000/50000 [==============================] - 2s 31us/step - loss: 0.7337 - acc: 0.7474 - val_loss: 0.7603 - val_acc: 0.7464
Epoch 3/50
50000/50000 [==============================] - 2s 31us/step - loss: 0.6792 - acc: 0.7640 - val_loss: 0.7280 - val_acc: 0.7549
Epoch 4/50
50000/50000 [==============================] - 2s 31us/step - loss: 0.6549 - acc: 0.7721 - val_loss: 0.7357 - val_acc: 0.7564
Epoch 5/50
50000/50000 [==============================] - 2s 31us/step - loss: 0.6397 - acc: 0.7779 - val_loss: 0.7141 - val_acc: 0.7642
Epoch 6/50
50000/50000 [==============================] - 2s 31us/step - loss: 0.6256 - acc: 0.7838 - val_loss: 0.7275 - val_acc: 0.7612
Epoch 7/50
50000/50000 [==============================] - 2s 31us/step - loss: 0.6171 - acc: 0.7866 - val_loss: 0.7301 - val_acc

50000/50000 [==============================] - 1s 30us/step - loss: 0.5006 - acc: 0.8279 - val_loss: 0.6620 - val_acc: 0.7825

Epoch 00049: ReduceLROnPlateau reducing learning rate to 2.000000138479437e-22.
Epoch 50/50
50000/50000 [==============================] - 1s 30us/step - loss: 0.5007 - acc: 0.8273 - val_loss: 0.6619 - val_acc: 0.7821


In [7]:
X = X_input = keras.layers.Input(X_train.shape[1:])
X = G(X)
X = M(X)
C = keras.Model(X_input, X)
C.compile('nadam', 'sparse_categorical_crossentropy', ['acc'])
C.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
model_1 (Model)              (None, 1472)              20024384  
_________________________________________________________________
model_2 (Model)              (None, 10)                17674     
Total params: 20,042,058
Trainable params: 14,730
Non-trainable params: 20,027,328
_________________________________________________________________


In [8]:
C.evaluate(X_test, Y_test, batch_size=128)

10000/10000 [==============================] - 18s 2ms/step


[0.6618641731262207, 0.7821]